%load_ext watermark
%watermark -a "Chibuzor Enyioko" -d -v -p numpy,pandas,matplotlib,seaborn,sklearn

# Project 2: Supervised Classification

This project uses python packages to perform different unsupervised learning methods on a given breast cancer and diabetes dataset.

## Part 1: Breast Cancer Dataset
### Importing the Data

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

# importing data sets
bc_training_data = pd.read_csv("cancer_training.csv")
bc_test_data = pd.read_csv("cancer_testing.csv")

# selecting features and target variable
X_train = bc_training_data.drop(columns=['class'])
y_train = bc_training_data['class']

X_test = bc_test_data.drop(columns=['class'])
y_test = bc_test_data['class']


### Problems
1. Identify which column(s) (both train and test) has/have missing values? Identify the ‘row id’s.
“Impute” them with “Average/Most Frequent” values.

In [87]:
from sklearn.impute import SimpleImputer

# Imputating missing values with most frequent value
imputer = SimpleImputer(strategy='most_frequent')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)

# Identifying columns with missing values
# training set

X_train.replace("?", np.nan, inplace=True)
missing_val_columns = X_train.isnull().sum()
print(f"In the training set, the missing columns are:\n{missing_val_columns[missing_val_columns > 0]}")

for col in missing_val_columns[missing_val_columns > 0].index:
    missing_rows = X_train[X_train[col].isnull()]
    print(f"Column '{col}' has missing values in rows")
    row_id = []
    for i in missing_rows['id']:
        row_id.append(i)
    print(f"Row ID's with missing value in column '{col}': {row_id}")

# test set
X_test.replace("?", np.nan, inplace=True)
missing_val_columns_test = X_test.isnull().sum()
print("\n"f"In the test set, the missing columns are:\n{missing_val_columns_test[missing_val_columns_test > 0]}")
for col in missing_val_columns_test[missing_val_columns_test > 0].index:
    missing_rows_test = X_test[X_test[col].isnull()]
    print(f"Column '{col}' has missing values in rows")
    row_id = []
    for i in missing_rows_test['id']:
        row_id.append(i)
    print(f"Row ID's with missing value in column '{col}': {row_id}")




In the training set, the missing columns are:
node-caps      3
breast-quad    1
dtype: int64
Column 'node-caps' has missing values in rows
Row ID's with missing value in column 'node-caps': [7, 64, 179]
Column 'breast-quad' has missing values in rows
Row ID's with missing value in column 'breast-quad': [155]

In the test set, the missing columns are:
node-caps    5
dtype: int64
Column 'node-caps' has missing values in rows
Row ID's with missing value in column 'node-caps': [21, 32, 51, 55, 72]


2. Calculate accuracy using each of these classifiers (up to 3 decimal places):

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
